In [11]:
import torch

In [12]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

In [13]:
data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = [raw_text[i - j - 1] for j in range(CONTEXT_SIZE)] + [raw_text[i + j + 1] for j in range(CONTEXT_SIZE)]
    target = raw_text[i]
    data.append((context, target))

In [14]:
vocab = set(raw_text)
vocab_size = len(vocab)
word_to_idx = {word:i for i, word in enumerate(vocab)}

In [15]:
word_to_idx

{'processes.': 0,
 'We': 1,
 'computers.': 2,
 'computer': 3,
 'our': 4,
 'other': 5,
 'directed': 6,
 'pattern': 7,
 'are': 8,
 'programs': 9,
 'a': 10,
 'In': 11,
 'spirits': 12,
 'conjure': 13,
 'that': 14,
 'The': 15,
 'process': 16,
 'effect,': 17,
 'inhabit': 18,
 'beings': 19,
 'program.': 20,
 'the': 21,
 'computational': 22,
 'spells.': 23,
 'of': 24,
 'direct': 25,
 'Computational': 26,
 'things': 27,
 'abstract': 28,
 'we': 29,
 'with': 30,
 'As': 31,
 'they': 32,
 'create': 33,
 'to': 34,
 'called': 35,
 'idea': 36,
 'evolution': 37,
 'manipulate': 38,
 'is': 39,
 'rules': 40,
 'about': 41,
 'study': 42,
 'evolve,': 43,
 'processes': 44,
 'by': 45,
 'data.': 46,
 'process.': 47,
 'People': 48}

In [16]:
class CBOW(torch.nn.Module):

    def __init__(self, vocab_size, embedding_size, context_size) -> None:
        super(CBOW, self).__init__()
        self.embeddinglayer = torch.nn.Embedding(vocab_size, embedding_size)
        self.linear1 = torch.nn.Linear(2 * context_size * embedding_size, 128)
        self.linaer2 = torch.nn.Linear(128, vocab_size)

    def forward(self, x):
        x = self.embeddinglayer(x).view(1, -1)
        x = self.linear1(torch.nn.functional.relu(x))
        x = self.linaer2(x)
        out = torch.nn.functional.log_softmax(x, dim=1)
        return out

In [17]:
cbow = CBOW(vocab_size=vocab_size, embedding_size=16, context_size=CONTEXT_SIZE)

In [18]:
losses = []

loss_fn = torch.nn.NLLLoss()
opt = torch.optim.SGD(cbow.parameters(), lr=1e-3)

In [19]:
for epoch in range(20):
    for x,y in data:
        cbow.zero_grad()
        x_tensor = torch.tensor([word_to_idx[i] for i in x], dtype=torch.long)
        pred = cbow(x_tensor)
        loss = loss_fn(pred, torch.tensor([word_to_idx[target]], dtype=torch.long))
        loss.backward()
        opt.step()
        losses.append(loss.item())

In [20]:
losses

[4.100916385650635,
 3.9330012798309326,
 4.649822235107422,
 4.066380023956299,
 4.233892917633057,
 4.431120872497559,
 4.193294525146484,
 4.319066047668457,
 4.484870910644531,
 3.9676361083984375,
 4.266733646392822,
 4.213759422302246,
 3.623493194580078,
 4.1785664558410645,
 3.925682783126831,
 4.230259418487549,
 3.900003433227539,
 4.130650997161865,
 3.9693305492401123,
 4.069216728210449,
 3.975330114364624,
 4.121794700622559,
 4.073111534118652,
 3.9946630001068115,
 4.417471408843994,
 3.9392473697662354,
 3.997938632965088,
 3.9915285110473633,
 3.6735050678253174,
 3.9562110900878906,
 3.78001070022583,
 3.9776105880737305,
 3.9836249351501465,
 3.884711265563965,
 4.224284648895264,
 4.139175891876221,
 3.7904367446899414,
 4.0238542556762695,
 3.6737022399902344,
 4.188233375549316,
 4.097197532653809,
 3.851916790008545,
 3.449598789215088,
 3.9765872955322266,
 3.93605899810791,
 3.801936626434326,
 3.8417680263519287,
 3.6751327514648438,
 3.7626888751983643,
 3.9